<a href="https://colab.research.google.com/github/azizmh36/Sentiment-Analysis-of-Airlines/blob/main/mohamedazizmouhli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python packages to manipulate files
import os
import shutil
import pathlib
from pathlib import Path
import datetime
# Tensorflow and Numpy packages
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Activation, Flatten, Dropout
import numpy as np

# Display related packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image
import PIL
import PIL.Image

%matplotlib inline

In [ ]:
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')
# #tf.config.set_visible_devices([], 'GPU')

In [ ]:
! git clone https://github.com/minus--/arabic-letters-tutorial.git

Cloning into 'arabic-letters-tutorial'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 50 (delta 25), reused 31 (delta 12), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [ ]:
! [ ! -d ./data ] && tar xvzf ./arabic-letters-tutorial/arabic_handwritten_data.tgz || echo "Dataset folder already exists"

In [ ]:
arabic_characters = ['alef', 'beh', 'teh', 'theh', 'jeem', 'hah', 'khah', 'dal', 'thal',
                    'reh', 'zain', 'seen', 'sheen', 'sad', 'dad', 'tah', 'zah', 'ain',
                    'ghain', 'feh', 'qaf', 'kaf', 'lam', 'meem', 'noon', 'heh', 'waw', 'yeh']

In [ ]:
logdir = os.path.join("./logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
%load_ext tensorboard

In [ ]:
batch_size = 32
img_height = 32
img_width = 32

def get_dataset(dataset_dir):
    
    def process_filename(file_path):
        label = tf.strings.regex_replace(input=file_path,pattern=r".+_label_(\d+)\.png", rewrite=r"\1")
        label = tf.strings.to_number(label, tf.int32)-1
        #label = tf.one_hot(label, depth=29)
        return label

    def process_img(file_path):

        img = tf.io.read_file(file_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, size=(32, 32))
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.cast(img, tf.float32) / 255.0
        return img
    
    data_dir = pathlib.Path(dataset_dir)
    file_list = [str(path.absolute()) for path in Path(data_dir).glob("*.png")]
    files_ds = tf.data.Dataset.from_tensor_slices((file_list))
    files_ds = files_ds.map(lambda x: (process_img(x), process_filename(x)))
    return files_ds

In [ ]:
train_dataset_path = "data/train_data"
test_dataset_path = "data/test_data"

train_ds = get_dataset(train_dataset_path).shuffle(buffer_size=batch_size*10).batch(batch_size)
valid_ds = get_dataset(test_dataset_path).batch(batch_size)


In [ ]:
Model=Sequential()
Model.add(Conv2D(64,(3,3),input_shape=(32,32,3)))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))

Model.add(Conv2D(128,(3,3)))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))

Model.add(Flatten())

Model.add(Dense(512))
Model.add(Activation('relu'))
Model.add(Dropout(0.5))
Model.add(Dense(28))
Model.add(Activation('softmax'))

Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
activation_1 (Activation)    (None, 13, 13, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [ ]:
Model.compile(
    # Optimizer
    optimizer=tf.keras.optimizers.RMSprop(),  
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # List of metrics to monitor
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=logdir,
        histogram_freq=0,  # How often to log histogram visualizations
        embeddings_freq=0,  # How often to log embedding visualizations
        update_freq="epoch",
    ) 
]

In [14]:
print("Fit model on training data")
history = Model.fit(train_ds, epochs = 50 ,validation_data=valid_ds, callbacks = callbacks)

Fit model on training data
Epoch 1/50
  1/420 [..............................] - ETA: 0s - loss: 3.3306 - sparse_categorical_accuracy: 0.0000e+00WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
420/420 [==============================] - 51s 122ms/step - loss: 3.1314 - sparse_categorical_accuracy: 0.2598 - val_loss: 2.9931 - val_sparse_categorical_accuracy: 0.4015
Epoch 2/50
420/420 [==============================] - 52s 124ms/step - loss: 2.9573 - sparse_categorical_accuracy: 0.4374 - val_loss: 2.8734 - val_sparse_categorical_accuracy: 0.5214
Epoch 3/50
420/420 [==============================] - 51s 121ms/step - loss: 2.8508 - sparse_categorical_accuracy: 0.5450 - val_loss: 2.7749 - val_sparse_categorical_accuracy: 0.6185
Epoch 4/50
420/420 [===================

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(32,32,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(29, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 16)        80        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        2080      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          8256      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 64)         

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [23]:
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=logdir,
        histogram_freq=0,  # How often to log histogram visualizations
        embeddings_freq=0,  # How often to log embedding visualizations
        update_freq="epoch",
    ) 
]

In [16]:
os.chdir("/content/data")
os.mkdir("class")
os.chdir('/content/data/class/')


In [17]:
for i in range (len(arabic_characters)):
    os.mkdir(arabic_characters[i])

In [18]:
os.chdir('/content/data/train_data/')

In [19]:
images = os.listdir()
for I in images:
    file_name,file_ext=os.path.splitext(I) 
    file_name= file_name.split('_')
    
    lab_nbr=file_name[-1]
    if lab_nbr=='1':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/alef')
    elif lab_nbr=='2':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/beh')
    elif lab_nbr=='3':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/teh')
    elif lab_nbr=='4':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/theh')
    elif lab_nbr=='5':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/jeem')
    elif lab_nbr=='6':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/hah')
    elif lab_nbr=='7':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/khah')
    elif lab_nbr=='8':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/dal')
    elif lab_nbr=='9':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/thal')
    elif lab_nbr=='10':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/reh')
    elif lab_nbr=='11':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/zain')
    elif lab_nbr=='12':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/seen')
    elif lab_nbr=='13':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/sheen')
    elif lab_nbr=='14':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/sad')
    elif lab_nbr=='15':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/dad')
    elif lab_nbr=='16':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/tah')
    elif lab_nbr=='17':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/zah')
    elif lab_nbr=='18':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/ain')
    elif lab_nbr=='19':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/ghain')
    elif lab_nbr=='20':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/feh')
    elif lab_nbr=='21':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/qaf')
    elif lab_nbr=='22':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/kaf')
    elif lab_nbr=='23':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/lam')
    elif lab_nbr=='24':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/meem')
    elif lab_nbr=='25':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/noon')
    elif lab_nbr=='26':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/heh')
    elif lab_nbr=='27':
        shutil.move('/content/data/train_data/'+I,'/content/data/class/waw')
    else:
        shutil.move('/content/data/train_data/'+I,'/content/data/class/yeh')
    

In [20]:
for i in range (len(arabic_characters)):
  print(len(os.listdir("/content/data/class/"+arabic_characters[i])))

480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
480
481


In [25]:
for i in range (len(arabic_characters)):
    os.mkdir(arabic_characters[i])

In [26]:
os.chdir('/content/data/train_data/')
os.chdir("/content/data")
os.mkdir("class_test")
os.chdir('/content/data/class_test/')

In [27]:
for i in range (len(arabic_characters)):
    os.mkdir(arabic_characters[i])

In [28]:
os.chdir('/content/data/test_data/')

In [29]:
images = os.listdir()
for I in images:
    file_name,file_ext=os.path.splitext(I) 
    file_name= file_name.split('_')
    lab_nbr=file_name[-1]
    if lab_nbr=='1':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/alef')
    elif lab_nbr=='2':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/beh')
    elif lab_nbr=='3':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/teh')
    elif lab_nbr=='4':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/theh')
    elif lab_nbr=='5':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/jeem')
    elif lab_nbr=='6':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/hah')
    elif lab_nbr=='7':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/khah')
    elif lab_nbr=='8':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/dal')
    elif lab_nbr=='9':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/thal')
    elif lab_nbr=='10':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/reh')
    elif lab_nbr=='11':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/zain')
    elif lab_nbr=='12':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/seen')
    elif lab_nbr=='13':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/sheen')
    elif lab_nbr=='14':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/sad')
    elif lab_nbr=='15':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/dad')
    elif lab_nbr=='16':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/tah')
    elif lab_nbr=='17':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/zah')
    elif lab_nbr=='18':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/ain')
    elif lab_nbr=='19':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/ghain')
    elif lab_nbr=='20':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/feh')
    elif lab_nbr=='21':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/qaf')
    elif lab_nbr=='22':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/kaf')
    elif lab_nbr=='23':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/lam')
    elif lab_nbr=='24':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/meem')
    elif lab_nbr=='25':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/noon')
    elif lab_nbr=='26':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/heh')
    elif lab_nbr=='27':
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/waw')
    else:
        shutil.move('/content/data/test_data/'+I,'/content/data/class_test/yeh')

In [30]:
for i in range (len(arabic_characters)):
  print(len(os.listdir("/content/data/class_test/"+arabic_characters[i])))

120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120


In [31]:
Train_ds = ImageDataGenerator(rescale=1./255,
                                   rotation_range=45 ,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='reflect', cval=125) 
Test_ds= ImageDataGenerator(rescale=1./255)

In [32]:
Train_ds=Train_ds.flow_from_directory(directory='/content/data/class/',
                                                batch_size=32,
                                                target_size=(32, 32),
                                                classes = arabic_characters,
                                                class_mode='categorical'
                                                )

Found 13440 images belonging to 28 classes.


In [33]:
Test_ds=Test_ds.flow_from_directory(directory='/content/data/class_test/', 
                                    batch_size=16,  
                                    target_size=(32, 32),
                                    class_mode='categorical')

Found 3360 images belonging to 28 classes.


In [34]:
Model=Sequential()
Model.add(Conv2D(64,(3,3),input_shape=(32,32,3)))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))

Model.add(Conv2D(128,(3,3)))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))
Model.add(Flatten())
Model.add(Dense(512))
Model.add(Activation('relu'))
Model.add(Dropout(0.5))
Model.add(Dense(28))
Model.add(Activation('softmax'))
Model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
activation_4 (Activation)    (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
activation_5 (Activation)    (None, 13, 13, 128)       0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)             

In [35]:
Model.compile(
    # Optimizer
    optimizer=tf.keras.optimizers.Adam(),  
    # Loss function to minimize
    loss='categorical_crossentropy',
    # List of metrics to monitor
    metrics=["accuracy"],
)

In [36]:
Model.fit(Train_ds,
          steps_per_epoch= int(13440/32),
          validation_data=Test_ds,
          
          epochs=50)

Epoch 1/50
420/420 [==============================] - 58s 138ms/step - loss: 3.0219 - accuracy: 0.1205 - val_loss: 5.3744 - val_accuracy: 0.0324
Epoch 2/50
 85/420 [=====>........................] - ETA: 44s - loss: 2.4572 - accuracy: 0.2574

KeyboardInterrupt: ignored